In [1]:
%load_ext autoreload
%autoreload 2
from __future__ import absolute_import

import torch

import torch.nn as nn
import torch.optim as optim

import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn.parameter import Parameter

import torch.cuda
import numpy as np

from qpth.qp import QPFunction

import matplotlib.pyplot as plt


class LSTMModel(nn.Module):
    def __init__(self, inputSize, nHidden, noutputs=3, numLayers=1):
        super(LSTMModel, self).__init__()
        
        self.cost = nn.MSELoss(size_average=False)
        self.noutputs = noutputs
        self.num_layers = numLayers
        self.inputSize = inputSize
        self.nHidden = nHidden
        # self.neunet = nn.Sequential()
        self.lstm1  = nn.LSTM(inputSize,nHidden,num_layers=numLayers)
        self.drop   = nn.Dropout(0.3)
        self.fc1 = nn.Linear(nHidden, noutputs)

    def forward(self, x):
        # Set initial states 
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.nHidden).cuda()) 
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.nHidden).cuda())
        
        # Forward propagate RNN
        out, _ = self.lstm1(x, (h0, c0))  
        
        # Decode hidden state of last time step
        out = self.fc1(out[:, -1, :])  
        return out

#hyperparams
inputSize = 1
nHidden   = 9
numLayers = 2
sequence_length = 9
num_epochs = 500
noutputs = 3

lstm = LSTMModel(inputSize, nHidden, noutputs=3, numLayers=1)
lstm.cuda()

# Loss and Optimizer
criterion = nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(lstm.parameters(), lr=0.1)

#images and labels
images = torch.Tensor(300, 9, 3)
labels = torch.Tensor(3)


# Train the Model
for epoch in range(num_epochs):
    for i in range(num_epochs):
        images = Variable(images.view(-1, sequence_length, inputSize)).cuda()  
        labels = Variable(labels).cuda()
        print images.size()
        print labels.size()
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = lstm(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

RuntimeError: cuda runtime error (6) : the launch timed out and was terminated at /home/lex/Documents/NNs/locuslab-pytorch/torch/lib/THC/generic/THCStorage.cu:66